### Notes

The models you can run here will be limited by what's implemented by the litellm library, because that's what managing the API calls.

You can see the models here: https://docs.litellm.ai/docs/



In [1]:
from functions import *

## Parameters

In [2]:
num_perturbations = 2
models_dict = {
    #'claude-2.1':  "ANTHROPIC", 
    'gpt-3.5-turbo-0301': "OPENAI"
               } 
csv_file_path = '../data/prompt_target_answer_pairs.csv'
similarity_model_name = 'sentence-transformers/paraphrase-mpnet-base-v2'
num_runs = 2
temperature = .9



In [3]:
prompts = [
    {"prompt": "What is the capital of France?"},
    {"prompt": "What is the boiling point of water?"}
]
instructions = "Please answer thoroughly and in the form of a poem: "
results = process_prompts(prompts, models_dict, num_perturbations=num_perturbations, num_runs=num_runs, is_file_path=False, perform_similarity=False, instructions =instructions, temperature=temperature)
  
results


2023-12-01 13:50:40,608 - INFO - Using direct list of prompts.
2023-12-01 13:50:41,390 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:43,946 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:47,058 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:47,768 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:49,745 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:52,754 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:53,258 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:55,405 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2023-12-01 13:50:58,936 - INFO - 

,model,original_prompt,response,temperature,actual_prompt
0,gpt-3.5-turbo-0301,What is the capital of France?,"The capital of France, oh don't you know,\nIs ...",0.7,What is the capital of France?
1,gpt-3.5-turbo-0301,What is the capital of France?,"Oh, curious one, let me tell you true, \nThe ...",0.7,I am curious to know the capital city of France.
2,gpt-3.5-turbo-0301,What is the capital of France?,"Amidst the rolling hills of France,\nLies a ci...",0.7,What is the capital of France?
3,gpt-3.5-turbo-0301,What is the capital of France?,"In the heart of France, a city stands tall,\nI...",0.7,I'm curious to know which city in France is co...
4,gpt-3.5-turbo-0301,What is the boiling point of water?,"The boiling point of water, oh so dear,\nIs a ...",0.7,What is the boiling point of water?
5,gpt-3.5-turbo-0301,What is the boiling point of water?,"Water, oh water, so clear and so pure,\nAt wha...",0.7,What is the temperature at which water begins ...
6,gpt-3.5-turbo-0301,What is the boiling point of water?,"Water, oh water, so pure and so clear\nThe boi...",0.7,What is the boiling point of water?
7,gpt-3.5-turbo-0301,What is the boiling point of water?,"When water's in a boiling state,\nIts temperat...",0.7,What is the temperature required for water to ...


In [5]:
results['response'].values

array([' Paris', ' Paris', ' 212 degrees Fahrenheit',
       ' 212 degrees Fahrenheit', 'Paris.', 'Paris.',
       '100 degrees Celsius or 212 degrees Fahrenheit.',
       '100 degrees Celsius (212 degrees Fahrenheit) at standard atmospheric pressure.'],
      dtype=object)

## Responses from LLMs

### With Similarity Scores

In [7]:
results ['similarity_score'] = calculate_similarity(results ['target_answer'].tolist(), results['response'].tolist(), similarity_model_name)
results


KeyError: 'target_answer'

### Best and Worst Response from Each Model/Prompt Combo

In [5]:
aggregate_results(responses_df)

,model,original_prompt,best_answer,best_similarity,worst_answer,worst_similarity
0,gpt-3.5-turbo-0301,What is the powerhouse of the cell?,The mitochondria is known as the energy factor...,0.839692,The primary source of energy production in the...,0.537239
1,gpt-3.5-turbo-0301,What is the basic unit of life?,The fundamental entity that makes up all livin...,0.834349,The smallest structural and functional unit of...,0.712358
2,gpt-3.5-turbo-0301,What do genes consist of?,Genes are composed of DNA (deoxyribonucleic ac...,0.815171,Genetic structures consist of DNA (deoxyribonu...,0.681154
3,gpt-3.5-turbo-0301,How do astroflora perform photosynthesis in lo...,"Astroflora, or plants that grow in space, have...",0.806800,"As an AI language model, I do not have informa...",0.703187
4,claude-2.1,What is the powerhouse of the cell?,The mitochondria are the intracellular struct...,0.804405,Aerobic respiration and most ATP synthesis ta...,0.486933
5,claude-2.1,What do genes consist of?,Genes are made up of DNA (deoxyribonucleic ac...,0.779261,The basic building blocks of genes are nucleo...,0.650256
6,gpt-3.5-turbo-0301,What particle stabilizes the membrane of hover...,"As an AI language model, I cannot find any spe...",0.756064,The particle is called dermaseptin.,0.332667
7,claude-2.1,What is the basic unit of life?,The most basic component that makes up all li...,0.754846,"There is no definitive ""simplest form of matt...",0.488756
8,claude-2.1,What particle stabilizes the membrane of hover...,"Unfortunately, I do not have enough context t...",0.714272,"Unfortunately, I do not have enough context t...",0.562627
9,claude-2.1,How do astroflora perform photosynthesis in lo...,"Unfortunately, I do not have enough context t...",0.632290,There are no known plants native to outer spa...,0.385661
